# Algoritmi Genetici

In [1]:
import numpy as np
from functools import reduce

__1. Sa se implementeze un algoritm genetic care sa calculeze maximul functiei :__

${f: [-20,20] x [-5,5] -> (0,\inf)}$

### ${f(x,y) = \left | \frac{10x^{2}y-5x^{2}-4y^{2}-x^{4}-2y^{4}}{2} \right |+1}$



${\bigstar }$ Parametrii algoritmului:

__n__- numarul de indivizi din populatie

__k__- numarul de gene, numarul de biti care intra in cadrul fiecarui individ

__c__- cromozom (sir de k biti)

__${p_{c}}$__-parametru al aplicatiei folosit pt alegerea cromozomilor care vor participa la incrucisare

__t__-numar aleatoriu folosit pt impartirea genelor parintilor catre cei doi copii

__${p_{m}}$__-parametru al aplicatiei folosit pt alegerea cromozomilor care sufera o mutatie

## Hiperparametrii

In [2]:
a_1 = -20
b_1 = 20
a_2 = -5
b_2 = 5

n = 1000 #100 cromozomi, indivizi
k = 50 #50 gene per cromozom
pc = 0.1
pm = 0.01
nr_generatii = 100 #generatii

## Crearea unei populatii

In [3]:
def f(x, y):
    """    
    :return: returneaza functia careia trebuie sa-i calculam maximul
    """
    
    return np.absolute((10*x**2*y - 5*x**2 - 4*y**2 - x**4 - 2*y**4) / 2) + 1

In [4]:
def setare_gene_biti(k: int) -> list:
    """    
    :param k: un integer care reprezinta numarul de biti, gene care intra in cadrul fiecarui individ, cromozom
    :return: returneaza o lista de cifre binare aleatoare de lungime egala cu lungimea setata prin hiperparametrul k
    """
    
    return np.random.randint(2,size=(1,k))

In [5]:
def setare_cromozomi(k: int,n: int) -> list:
    """    
    :param n: un integer care reprezinta numarul de cromozomi, de indivizi din populatie
    :param k: un integer care reprezinta numarul de biti, gene care intra in cadrul fiecarui individ, cromozom
    :return: returneaza o lista de cromozomi initiali
    """
    
    cromozomi = []
    for i in range(n):
        cromozomi.append(setare_gene_biti(k))
    #print(cromozomi)
    return cromozomi

In [6]:
def v(c :list) -> int:
    """    
    Functia face o transformarea in baza 10.
    :param c: cromozom
    :return: returneaza genele cromozomului in baza 10
    """
    
    powers_two = (2.0**np.arange(k)[::-1]).astype(np.int64)
    decimal_value = np.dot(c,powers_two)
    return decimal_value
    

In [7]:
def val_interval(c :list, a: int, b: int, k: int):
    """    
    :param c: cromozom
    :return: returneaza valoarea asociata cromozomilor in intervalul [a,b]
    """
    
    return a+v(c)*((b-a)/(2**k-1))

## Functii ajutatoare 

In [8]:
def in_interval(array: list, valoare: float) -> int:
    """
    Functie ajutatoare pentru pasul de selectie. In aceasta functie se alege indexul cromozomilor care vor fi 
    pastrati pentru urmatorii pasi. 
    :param array: probabilitate cumulativa de selectie
    :param valoare: o valoare din lista de valori random 
    :return: indexul celor mai buni indivizi
    """
    
    veriable = list(array.copy())
    veriable.append(valoare)
    veriable = sorted(veriable)
    return veriable.index(valoare)

In [9]:
def cross(cromozomi_incrucisare: list, t: int) -> list:
    """
    Functie ajutatoare pentru pasul de incrucisare. In aceasta functie parintii alesi se vor incrucisa si vor rezulta noii 
    cromozomi, copii.
    :param cromozomi_incrucisare: lista de cromozomi, de parinti alesi pentru incrucisare
    :param t: numar aleatoriu folosit pt impartirea genelor parintilor catre cei doi copii
    :return: lista cromozomilor, copiilor dupa incrucisarea parintilor
    """
    
    cromozomii_noi = []
    tata = reduce(lambda x,y: x+y,cromozomi_incrucisare[0])
    mama = reduce(lambda x,y: x+y,cromozomi_incrucisare[1])
    
    for j in range(0,len(cromozomi_incrucisare)-1):
        copil1 = np.concatenate((tata[:t] , mama[t:]), axis=None)
        copil2 = np.concatenate((mama[:t] , tata[t:]), axis=None)
        cromozomii_noi.append(copil1)
        
        tata = copil2
        if j!=len(cromozomi_incrucisare)-2:
            mama = reduce(lambda x,y: x+y,cromozomi_incrucisare[j+2])
        
    cromozomii_noi.append(copil2)
    return cromozomii_noi

In [10]:
def flip(c: int) -> int:
    """
    Functie ajutatoare pentru pasul de mutatie. Cand gena unui cromozom este 0 prin mutatie devin 1 si invers.
    :param c: o gena a unui cromozom 
    :return: returneaza 1 sau 0
    """

    return 1 if (c == 0) else 0

## Algoritmul Genetic

In [11]:
def selectia(functie: list, cromozomi: list) -> list:
    """
    :param cromozomi: lista de cromozomi
    :param functie: valorile asociate functiei
    :return: returneaza valoarea noilor cromozomi dupa selectie
    """
    
    suma = np.sum(functie)
    probabilitate_selectie = functie/suma
    probabilitate_cumulativa_selectie = np.cumsum(probabilitate_selectie)
    rand_selection = np.random.uniform(0, 1, n) 
    cromozomi_noi = [cromozomi[in_interval(probabilitate_cumulativa_selectie,rand_selection[i])] for i in range(0,n)]
    return cromozomi_noi

In [12]:
def incrucisare(cromozomi: list) -> list:
    """
    :param cromozomi: lista de cromozomi
    :return: returneaza valoarea noilor cromozomi dupa incrucisare
    """
    
    rand_incrucisare = np.random.uniform(0, 1, n) 
    index_incrucisare = reduce(lambda x,y: x+y,np.where(rand_incrucisare < pc))
    if len(index_incrucisare) % 2 != 0 :
        index_incrucisare = index_incrucisare[:-1]

    cromozomi_incrucisare = [cromozomi[i] for i in index_incrucisare]
    t = np.random.randint(1, k-1)
    cromozomi_incrucisare = cross(cromozomi_incrucisare,t)
    
    return cromozomi_incrucisare

In [13]:
def mutatie(cromozomi: list) -> list:
    """
    :param cromozomi: lista de cromozomi
    :return: returneaza valoarea noilor cromozomi dupa mutatie
    """
    
    for i in range(0,len(cromozomi)):
        rand_mutation = np.random.uniform(0, 1, k) 
        index_mutatie = reduce(lambda x,y: x+y,np.where(rand_mutation < pm))
        for j in index_mutatie:
            cromozomi[i][j] = flip(cromozomi[i][j])
    return cromozomi

In [14]:
def algoritm_genetic(k: int, n: int):
    """   
    Functia construieste un fisier cu valorile cromozomilor x si y si cu valoarea functiei corespunzatoare acestor cromozomi
    dupa aplicarea algoritmului genetic.
    Aceste valori se introduc in fisierul text pentru fiecare generatie.
    :param k: un integer care reprezinta numarul de biti, gene care intra in cadrul fiecarui individ, cromozom
    :param n: un integer care reprezinta numarul de cromozomi, de indivizi din populatie
    """
    
    file = open("fisier_iesire_alg_gen.txt",'w') 
    cromozomi = setare_cromozomi(k,n)
    for generatia in range(0,nr_generatii):
        file.writelines('#inceput date generatie %d \n' %generatia)
    
        x = val_interval(cromozomi, a_1, b_1, k)
        y = val_interval(cromozomi, a_2, b_2, k)
        functie = f(x,y)
        
        for i in range(0,len(cromozomi)):
            file.writelines('cromozom: x=%f y=%f \n' % (x[i],y[i]))
            file.writelines('valoare: %f \n' %functie[i])
       
        cromozomii_dupa_selectie = selectia(functie, cromozomi)
        cromozomii_dupa_incrucisare = incrucisare(np.array(cromozomii_dupa_selectie))
        cromozomii_dupa_mutatie = mutatie(cromozomii_dupa_incrucisare)
        for i in range(0,len(cromozomii_dupa_mutatie)):
            cromozomii_dupa_mutatie[i] = cromozomii_dupa_mutatie[i].reshape(-1,len(cromozomii_dupa_mutatie[i]))
        cromozomi= cromozomii_dupa_mutatie
        
        file.writelines('#sfarsit date generatie %d \n' %generatia)
        
    file.close()
          
algoritm_genetic(k,n)